In [1]:
import requests
import pandas as pd

# RemoteOk data
def fetch_remoteok_jobs():
    url = "https://remoteok.com/api"
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=headers)
    jobs_json = response.json()

    # Skip the first item as it is metadata
    jobs = jobs_json[1:]

    job_list = []
    for job in jobs:
        job_list.append({
            "title": job.get("position"),
            "company": job.get("company"),
            "location": job.get("location"),
            "tags": ", ".join(job.get("tags", [])),
            "date_posted": job.get("date"),
            "salary": job.get("salary"),
            "url": job.get("url")
        })

    df = pd.DataFrame(job_list)
    df.to_csv("data/remoteok_jobs.csv", index=False)
    print(f"Fetched {len(df)} jobs from API.")
    return df

df_jobs = fetch_remoteok_jobs()

Fetched 94 jobs from API.


In [2]:
import requests
import pandas as pd

def scrape_linkedin_jobs():
    url = "https://linkedin-job-search-api.p.rapidapi.com/active-jb-7d?limit=10&offset=0&title_filter=%22Data%20Engineer%22&location_filter=%22United%20States%22%20OR%20%22United%20Kingdom%22"

    headers = {
        "X-RapidAPI-Key": "28ee349fb7mshe0f8b3bc34d8332p1299efjsnbaa7eb44c993",  # Replace with your real key
        "X-RapidAPI-Host": "linkedin-job-search-api.p.rapidapi.com"
    }

    querystring = {
        "limit": "75",
        "offset": "0",
        "title_filter": "Data Engineer",
        "location_filter": "United States" 
    }

    response = requests.get(url, headers=headers, params=querystring)

    try:
        job_data = response.json()
    except Exception as e:
        print("Failed to decode JSON:", e)
        print("Raw response:", response.text[:500])
        return pd.DataFrame()

    if not job_data:
        print("No jobs found.")
        return pd.DataFrame()

    jobs = []
    for job in job_data:
        jobs.append({
            "title": job.get("title"),
            "company": job.get("organization"),
            "location": job.get("locations_raw", [{}])[0].get("address", {}).get("addressLocality", ""),
            "tags": "",
            "date_posted": job.get("date_posted"),
            "source": "LinkedIn (RapidAPI)"
        })

    df = pd.DataFrame(jobs)
    df.to_csv("data/linkedin_jobs.csv", index=False)
    print(f"Fetched {len(df)} jobs from LinkedIN (RapidAPI).")
    return df

if __name__ == "__main__":
    scrape_linkedin_jobs()


Fetched 75 jobs from LinkedIN (RapidAPI).


In [3]:
import pandas as pd

# Load your job datasets
remoteok = pd.read_csv("remoteok_jobs.csv")
linkedin = pd.read_csv("linkedin_jobs.csv")

# Add missing 'source' column to RemoteOK
remoteok["source"] = "RemoteOK"

# Standardize columns
required_cols = ["title", "company", "location", "tags", "date_posted", "source"]
remoteok = remoteok[required_cols]
linkedin = linkedin[required_cols]

# Combine
combined = pd.concat([remoteok, linkedin], ignore_index=True)
combined.to_csv("data/combined_raw_jobs.csv", index=False)
print(f"Combined dataset saved: {len(combined)} rows.")


Combined dataset saved: 169 rows.


In [5]:
import pandas as pd
from pandas import Timestamp

def clean_job_data(input_path="data/combined_raw_jobs.csv", output_path="data/clean_jobs.csv"):
    df = pd.read_csv(input_path)

    # Drop jobs with no title or company
    df = df.dropna(subset=["title", "company"])

    # Clean column names
    df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")

    # Convert tags to list
    df["tags"] = df["tags"].fillna("").apply(lambda x: x.split(",") if isinstance(x, str) else [])

    # Split by source
    remoteok_df = df[df["source"] == "RemoteOK"].copy()
    linkedin_df = df[df["source"].str.contains("LinkedIn", case=False, na=False)].copy()

    # --- Parse date_posted independently ---
    remoteok_df["date_posted"] = pd.to_datetime(remoteok_df["date_posted"].astype(str), errors="coerce")
    linkedin_df["date_posted"] = pd.to_datetime(linkedin_df["date_posted"].astype(str), errors="coerce")

    # Drop rows only if parsing truly failed in each subset
    remoteok_df = remoteok_df.dropna(subset=["date_posted"])
    linkedin_df = linkedin_df.dropna(subset=["date_posted"])

    # Combine
    combined = pd.concat([remoteok_df, linkedin_df], ignore_index=True)

    # Ensure all datetime entries are UTC and tz-aware
    combined["date_posted"] = pd.to_datetime(combined["date_posted"], utc=True)

    # Now subtract using same timezone
    today = pd.Timestamp.now(tz="UTC")
    combined["days_since_posted"] = (today - combined["date_posted"]).dt.days

    # Save final version
    combined.to_csv(output_path, index=False)
    print(f"Cleaned dataset saved to {output_path} with {len(combined)} rows.")

    return combined

if __name__ == "__main__":
    clean_job_data()


Cleaned dataset saved to data/clean_jobs.csv with 169 rows.
